In [1]:
from scipy.io.wavfile import read as read_wav
from IPython.display import Audio
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn as skl
import librosa as lr
import numpy as np
import logging
import random
import scipy
import os

In [2]:
np.random.seed(5)
logging.warnings.filterwarnings('ignore')

In [3]:
dataset_root = '..'

### Genre Dictionary:

In [4]:
genres = {
    'pop': 0,
    'reggae': 1
}

### Hyper Parameters:

In [5]:
hparams = {
    'samplerate': 22050,
    'seq_length': 256,
    'hop_length': 512,
    'fft_window': 2048,
    'num_classes': len(genres)
}

### Extract Features:

In [6]:
def extract_features(filepath, hparams):
    
    audio_data_array = lr.load(filepath, sr=hparams['samplerate'], mono=True)
    
    # normalized = lr.util.normalize(audio_data_array[0])
    
    features = [
        
        # MFCC Features:
        lr.feature.mfcc(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        ),

        # Chroma STFT:
        lr.feature.chroma_stft(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        ),
        
        # Spectral Centroid:
        lr.feature.spectral_centroid(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        ),
        
        # Spectral Contrast:
        lr.feature.spectral_contrast(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        )
    ]
    
    return np.vstack(features)

### Pop Class:

In [7]:
class_1 = [
    'pop'
]

### Reggae Class:

In [8]:
class_2 = [
    'reggae'
]

### Allocating Arrays to Store Dataset:

In [9]:
train_data_x = np.zeros(shape=(5 * 140, 40, hparams['seq_length']))
val_data_x = np.zeros(shape=(5 * 40, 40, hparams['seq_length']))
test_data_x = np.zeros(shape=(5 * 20, 40, hparams['seq_length']))

train_data_y = np.zeros(shape=(5 * 140, 2))
val_data_y = np.zeros(shape=(5 * 40, 2))
test_data_y = np.zeros(shape=(5 * 20, 2))

### Train Dataset:

In [10]:
preprocess = True

if preprocess == True:

    train_data_counter = 0
    val_data_counter = 0
    test_data_counter = 0

    for genre in class_1:

        print('Genre:', genre, 'Processed.')
        
        m = hparams['seq_length']

        filelist = os.listdir(dataset_root + '/dataset/' + genre)
        
        for file in filelist[0:70]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                train_data_x[train_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                train_data_y[train_data_counter] = tf.keras.utils.to_categorical(0, num_classes=2)
                train_data_counter += 1
        
        for file in filelist[70:90]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                val_data_x[val_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                val_data_y[val_data_counter] = tf.keras.utils.to_categorical(0, num_classes=2)
                val_data_counter += 1
                
        for file in filelist[90:100]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                test_data_x[test_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                test_data_y[test_data_counter] = tf.keras.utils.to_categorical(0, num_classes=2)
                test_data_counter += 1
        

    for genre in class_2:

        print('Genre:', genre, 'Processed.')
        
        m = hparams['seq_length']

        filelist = os.listdir(dataset_root + '/dataset/' + genre)
        
        for file in filelist[0:70]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                train_data_x[train_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                train_data_y[train_data_counter] = tf.keras.utils.to_categorical(1, num_classes=2)
                train_data_counter += 1
        
        for file in filelist[70:90]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                val_data_x[val_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                val_data_y[val_data_counter] = tf.keras.utils.to_categorical(1, num_classes=2)
                val_data_counter += 1
                
        for file in filelist[90:100]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                test_data_x[test_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                test_data_y[test_data_counter] = tf.keras.utils.to_categorical(1, num_classes=2)
                test_data_counter += 1
            
    if 'prepared' not in os.listdir('.'):
        os.mkdir('prepared')
    np.save('./prepared/train_data_x.npy', train_data_x, allow_pickle=False)
    np.save('./prepared/train_data_y.npy', train_data_y, allow_pickle=False)
    np.save('./prepared/val_data_x.npy', val_data_x, allow_pickle=False)
    np.save('./prepared/val_data_y.npy', val_data_y, allow_pickle=False)
    np.save('./prepared/test_data_x.npy', test_data_x, allow_pickle=False)
    np.save('./prepared/test_data_y.npy', test_data_y, allow_pickle=False)
else:
    train_data_x = np.load('./prepared/train_data_x.npy', allow_pickle=False)
    train_data_y = np.load('./prepared/train_data_y.npy', allow_pickle=False)
    val_data_x = np.load('./prepared/val_data_x.npy', allow_pickle=False)
    val_data_y = np.load('./prepared/val_data_y.npy', allow_pickle=False)
    test_data_x = np.load('./prepared/test_data_x.npy', allow_pickle=False)
    test_data_y = np.load('./prepared/test_data_y.npy', allow_pickle=False)

Genre: pop Processed.
Genre: reggae Processed.


In [11]:
train_data_x  = np.moveaxis(train_data_x, 1, 2)
val_data_x    = np.moveaxis(val_data_x,   1, 2)
test_data_x   = np.moveaxis(test_data_x,  1, 2)

### Model Creation and Training:

In [12]:
class GenreClassifierModel(tf.keras.Model):
    
    def __init__(self, l1, l2):
        
        super().__init__()
        
        self.L1 = tf.keras.layers.LSTM(l1, return_sequences=True)
        self.L2 = tf.keras.layers.LSTM(l2, return_sequences=False)
        self.L3 = tf.keras.layers.Dense(units=hparams['num_classes'], activation="softmax")
        
    def call(self, inputs):
        self.x = self.L1(inputs)
        self.x = self.L2(self.x)
        self.x = self.L3(self.x)
        return self.x

### Training Model:

In [13]:
def create_model(config):
    
    l1 = config['L1']
    l2 = config['L2']
    
    # Model Definition:
    model = GenreClassifierModel(l1, l2)
    
    # Model Compilation:
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [14]:
model = create_model({
    'L1': 64,
    'L2': 32
})

In [15]:
hist = model.fit(train_data_x, train_data_y, epochs=200)

Epoch 1/200
22/22 [==============================] - 0s 16ms/step - loss: 0.5846 - accuracy: 0.7114
Epoch 2/200
22/22 [==============================] - 0s 16ms/step - loss: 0.4564 - accuracy: 0.8014
Epoch 3/200
22/22 [==============================] - 0s 16ms/step - loss: 0.4503 - accuracy: 0.8029
Epoch 4/200
22/22 [==============================] - 0s 16ms/step - loss: 0.4207 - accuracy: 0.8243
Epoch 5/200
22/22 [==============================] - 0s 16ms/step - loss: 0.4059 - accuracy: 0.8229
Epoch 6/200
22/22 [==============================] - 0s 16ms/step - loss: 0.3888 - accuracy: 0.8400
Epoch 7/200
22/22 [==============================] - 0s 16ms/step - loss: 0.3643 - accuracy: 0.8543
Epoch 8/200
22/22 [==============================] - 0s 16ms/step - loss: 0.3715 - accuracy: 0.8457
Epoch 9/200
22/22 [==============================] - 0s 16ms/step - loss: 0.3422 - accuracy: 0.8500
Epoch 10/200
22/22 [==============================] - 0s 16ms/step - loss: 0.3475 - accuracy: 0.8600

22/22 [==============================] - 0s 17ms/step - loss: 0.1691 - accuracy: 0.9386
Epoch 83/200
22/22 [==============================] - 0s 17ms/step - loss: 0.1813 - accuracy: 0.9400
Epoch 84/200
22/22 [==============================] - 0s 18ms/step - loss: 0.1676 - accuracy: 0.9400
Epoch 85/200
22/22 [==============================] - 0s 17ms/step - loss: 0.1193 - accuracy: 0.9529
Epoch 86/200
22/22 [==============================] - 0s 17ms/step - loss: 0.1618 - accuracy: 0.9300
Epoch 87/200
22/22 [==============================] - 0s 17ms/step - loss: 0.1326 - accuracy: 0.9457
Epoch 88/200
22/22 [==============================] - 0s 17ms/step - loss: 0.1671 - accuracy: 0.9314
Epoch 89/200
22/22 [==============================] - 0s 17ms/step - loss: 0.1329 - accuracy: 0.9471
Epoch 90/200
22/22 [==============================] - 0s 17ms/step - loss: 0.0982 - accuracy: 0.9671
Epoch 91/200
22/22 [==============================] - 0s 17ms/step - loss: 0.1266 - accuracy: 0.9586
Epo

22/22 [==============================] - 0s 17ms/step - loss: 0.0837 - accuracy: 0.9743
Epoch 163/200
22/22 [==============================] - 0s 19ms/step - loss: 0.1003 - accuracy: 0.9771
Epoch 164/200
22/22 [==============================] - 0s 17ms/step - loss: 0.0311 - accuracy: 0.9871
Epoch 165/200
22/22 [==============================] - 0s 16ms/step - loss: 0.0299 - accuracy: 0.9871
Epoch 166/200
22/22 [==============================] - 0s 18ms/step - loss: 0.1072 - accuracy: 0.9671
Epoch 167/200
22/22 [==============================] - 0s 17ms/step - loss: 0.0528 - accuracy: 0.9871
Epoch 168/200
22/22 [==============================] - 0s 17ms/step - loss: 0.0544 - accuracy: 0.9814
Epoch 169/200
22/22 [==============================] - 0s 17ms/step - loss: 0.0680 - accuracy: 0.9743
Epoch 170/200
22/22 [==============================] - 0s 17ms/step - loss: 0.0215 - accuracy: 0.9929
Epoch 171/200
22/22 [==============================] - 0s 18ms/step - loss: 0.0694 - accuracy: 0

In [16]:
model.evaluate(val_data_x, val_data_y)

7/7 [==============================] - 0s 9ms/step - loss: 0.5886 - accuracy: 0.8750


[0.5886356830596924, 0.875]

In [17]:
model.save('./models/model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./models/model/assets
